In [75]:
# Se importan las librerías necesarias.

import pandas as pd, numpy as np, math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Se construirá un sistema de recomendación item-item, por lo que se utilizará el dataset "steam_games"

In [76]:
steam_games = pd.read_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\steam_games.parquet")

steam_games.head()

,publisher,genres,app_name,title,release_date,tags,specs,price,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",0.00,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",0.00,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,767400,彼岸领域
4,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]",3.99,772540,Trickjump Games Ltd


In [77]:
steam_games = steam_games.drop(steam_games.tail(int(steam_games.shape[0]/100*85)).index)

steam_games.head()

,publisher,genres,app_name,title,release_date,tags,specs,price,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",0.00,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",0.00,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,767400,彼岸领域
4,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,2018-01-04,"[Action, Adventure, Simulation, FPS, Shooter, ...","[Single-player, Steam Achievements]",3.99,772540,Trickjump Games Ltd


In [78]:
def SortList(l:list)->list:
    l.sort()
    return ",".join(l)

steam_games['tags'] = steam_games['tags'].apply(SortList)
# steam_games['specs'] = steam_games['specs'].apply(SortList)

steam_games.head()

,publisher,genres,app_name,title,release_date,tags,specs,price,id,developer
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"Action,Casual,Indie,Simulation,Strategy",[Single-player],4.99,761140,Kotoshiro
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"2D,Board Game,Card Game,Character Customizatio...","[Single-player, Multi-player, Online Multi-Pla...",0.00,643980,Secret Level SRL
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"Casual,Free to Play,Indie,Multiplayer,Simulati...","[Single-player, Multi-player, Online Multi-Pla...",0.00,670290,Poolians.com
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"Action,Adventure,Casual",[Single-player],0.99,767400,彼岸领域
4,Trickjump Games Ltd,"[Action, Adventure, Simulation]",Battle Royale Trainer,Battle Royale Trainer,2018-01-04,"Action,Adventure,FPS,Shooter,Simulation,Sniper...","[Single-player, Steam Achievements]",3.99,772540,Trickjump Games Ltd


In [79]:
vectorizer = TfidfVectorizer()

tags_vec = vectorizer.fit_transform(steam_games['tags']).toarray()
# developer_vec = vectorizer.fit_transform(steam_games['developer']).toarray()
# specs_vec = vectorizer.fit_transform(steam_games['specs']).toarray()

In [80]:
features = np.column_stack([tags_vec,steam_games['price']])

similities = cosine_similarity(features)

In [81]:
sim = pd.DataFrame(similities)

In [82]:
sim.to_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\Apis\recomendation\cosine_sim.parquet",compression='snappy')

In [83]:
# sim_parts = [sim.iloc[i:i+math.ceil(sim.shape[0]/10)] for i in range(0, len(sim), math.ceil(sim.shape[0]/10))]

In [84]:
# count = 1

# for i in sim_parts:
#     subparts = [i.iloc[j:j+math.ceil(i.shape[0]/6)] for j in range(0, len(i), math.ceil(i.shape[0]/6))]
#     for k in range(6):
#         subparts[k].to_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\Apis\recomendation\cosine_sim\cosine{}\cosine{}_{}".format(count,count,k+1),compression='gzip')
#     count+=1


In [85]:
steam_games = pd.read_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\steam_games.parquet",columns=['id','title'])


steam_games.to_parquet(r"C:\Users\roylo\OneDrive\Documentos\Data Science\Proyectos Individuales\Machine Learning Operations\Datasets\Apis\recomendation\games.parquet",compression='snappy')